In [1]:
import pandas as pd
import numpy as np
import hashlib
from collections.abc import Iterable
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn import metrics
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
import time

# define gvars
SEED = 1080
VCHUNK = 64

# import from csv to dataframe
df = pd.read_csv('wnl/WNL_TLS_Dataset_ECH.csv', sep='\t')



In [2]:
df.head()

#df.Label.unique()

,Label,ClientHello,ServerHello
0,AppleMusic,"22,3,1,0,166,1,0,0,162,3,3,0,0,0,0,147,162,126...","22,3,3,0,122,2,0,0,118,3,3,0,0,0,0,219,104,67,..."
1,AppleMusic,"22,3,1,1,213,1,0,1,209,3,3,0,0,0,0,153,108,195...","22,3,3,0,128,2,0,0,124,3,3,0,0,0,0,174,105,135..."
2,AppleMusic,"22,3,1,0,166,1,0,0,162,3,3,0,0,0,0,72,145,151,...","22,3,3,0,122,2,0,0,118,3,3,0,0,0,0,197,217,247..."
3,AppleMusic,"22,3,1,1,213,1,0,1,209,3,3,0,0,0,0,14,243,79,3...","22,3,3,0,128,2,0,0,124,3,3,0,0,0,0,73,224,90,8..."
4,AppleMusic,"22,3,1,1,213,1,0,1,209,3,3,0,0,0,0,24,22,171,9...","22,3,3,0,128,2,0,0,124,3,3,0,0,0,0,244,91,185,..."


In [3]:
'''https://tls13.xargs.org/#client-hello/annotated'''

def flatten(L):
    '''flattens a list of nested list of arbitrary depth into a single concatenated list'''
    for x in L:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x

def padding(A, size):
    '''pads a given list to a given length'''
    t = size - len(A)

    return np.pad(A, pad_width=(0, t), mode='constant')

def parse_CHP(p):
    '''
    implements the Shamsimukhametov et al. bytes recomposition algorithm
    builds a fixed length feature vector from clienthello message
    '''
    T = []
    L = []
    V = []

    handshake_len = sum(p[3:5])
    clienthello_len = sum(p[6:9])
    sid_len = p[43]
    i = 44+sid_len

    sid = p[44:i]
    ciphersuite_len = sum(p[i:i+2])
    ciphersuite = p[i+2:i+2+ciphersuite_len]
    i += 4 + ciphersuite_len

    ext_len = sum(p[i:i+2])
    i = i+2
    
    T.append(padding(sid, 32))
    L.append(handshake_len)
    L.append(clienthello_len)
    L.append(sid_len)
    L.append(ciphersuite_len)
    L.append(ext_len)
    V.append(ciphersuite)
    
    end = i + ext_len
    count = 0   

    while (i < len(p)):

        n_ext_id = sum(p[i:i+2])
        i+=2
        
        n_ext_len = sum(p[i:i+2])
        i+=2

        T.append(n_ext_id)
        L.append(n_ext_len)
        V.append(p[i:i+n_ext_len])
        i+=n_ext_len

    T = padding(list(flatten(T)),40)
    L = padding(list(flatten(L)),13)
    V = list(flatten(V))
    VC = V[:VCHUNK]

    return np.concatenate((T,L,VC))

def parse_SHP(p):
    '''
    implements the Shamsimukhametov et al. bytes recomposition algorithm
    builds a fixed length feature vector from serverhello message
    '''
    T = []
    L = []
    V = []

    handshake_len = sum(p[3:5])
    serverhello_len = sum(p[6:9])
    sid_len = p[43]
    i = 44+sid_len

    sid = p[44:i]
    ciphersuite = sum(p[i:i+2])
    i += 3

    ext_len = sum(p[i:i+2])
    i = i+2
    
    T.append(padding(sid, 32))
    T.append(ciphersuite)
    L.append(handshake_len)
    L.append(serverhello_len)
    L.append(sid_len)
    L.append(ext_len)

    end = i + ext_len
    count = 0    
    
    while (i < len(p)):

        n_ext_id = sum(p[i:i+2])
        i+=2
        
        n_ext_len = sum(p[i:i+2])
        i+=2

        T.append(n_ext_id)
        L.append(n_ext_len)
        V.append(p[i:i+n_ext_len])
        i+=n_ext_len

    T = padding(list(flatten(T)),40)
    L = padding(list(flatten(L)),10)
    V = list(flatten(V))
    VC = V[:VCHUNK]

    return np.concatenate((T,L,VC))

def parse_payload(CH,SH):
    '''builds a concatenated fixed length feature vector of recomposed bytes'''
    chp  = np.fromstring(CH, dtype=int, sep=',')
    shp  = np.fromstring(SH, dtype=int, sep=',')
    chv = parse_CHP(chp)
    shv = parse_SHP(shp)
    return np.concatenate((chv,shv))

In [4]:
# Transform according to recomposition algorithm
df['X'] = df.apply(lambda row : parse_payload(row['ClientHello'], row['ServerHello']), axis=1)
sample = df.X[0]
print(sample)
print(len(sample))

[132 119 186 235 186   2 164 141 185  92  21 164  31 142 242 106 128 185
  45  92  59 161 173 181  15 236 118 178 145  29 148 199  51  43   0   0
   0   0   0   0 166 162  32  34  55  38   9   0   0   0   0   0   0  19
   1  19   2  19   3 192  43 192  44 204 169 192  47 192  48 204 168 192
   9 192  10 192  19 192  20   0 156   0 157   0  47   0  53   0  36   0
  29   0  32 128  70 186 232  48 147  53 217 219 145  14  93 217 193 220
 251 228 172 113 190 211  40 106  94 132 119 186 235 186   2 164 141 185
  92  21 164  31 142 242 106 128 185  45  92  59 161 173 181  15 236 118
 178 145  29 148 199  20  43  51   0   0   0   0   0 122 118  32  46   2
  36   0   0   0   0   3   4   0  29   0  32 179 169  38  12  73  68  45
 171  35  59  24  99  89 181  22  35  31 149 206  44 227 135  92 156 245
 202  29  57 124  67 142  93]
205


In [5]:
# Encode classes into integers as terget variable vector y
labels = df.Label.unique()
df['Label'] = df['Label'].astype('category')
df['target'] = df['Label'].cat.codes
y = df['target'].to_numpy()

In [6]:
# expand array to proper dims
dfx = pd.DataFrame(df['X'].tolist()).add_prefix("x")
X = dfx.to_numpy(dtype=np.int16)

In [7]:
# split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=SEED, shuffle=True, stratify=y)

In [8]:
# build model using optimized params
dtc_opt_model = DecisionTreeClassifier(   criterion='entropy', 
                                    max_depth=8, 
                                    max_features=None, 
                                    min_samples_leaf=5, 
                                    min_samples_split=2,                                     
                                    splitter='random',
                                    random_state=SEED)

ada_opt_model = AdaBoostClassifier( base_estimator=dtc_opt_model, 
                                    learning_rate=1.0, 
                                    n_estimators=190, 
                                    random_state=SEED)
# start timing
start = time.time()
# fit the ada boost
ada_opt_model.fit(X_train,y_train)
# stop timing
end = time.time()

In [9]:
# Make predictions on our test data
y_pred = ada_opt_model.predict(X_test)

In [10]:
# Evaluate the models performance (I hope this isnt garbage)
print(classification_report(y_test, y_pred, target_names=labels, digits=4))

              precision    recall  f1-score   support

  AppleMusic     0.9038    1.0000    0.9495        47
         Web     0.8197    0.7692    0.7937        65
   Kinopoisk     1.0000    1.0000    1.0000        27
LiveFacebook     1.0000    1.0000    1.0000        27
 LiveYouTube     0.8125    0.9905    0.8927       105
     Netflix     0.9111    0.9111    0.9111        45
  PrimeVideo     0.9706    0.9706    0.9706        68
  SoundCloud     0.9344    0.9661    0.9500        59
     Spotify     1.0000    0.7391    0.8500        23
       Vimeo     0.9321    0.8110    0.8674       254
 YandexMusic     0.7864    0.8901    0.8351        91
     YouTube     0.9841    1.0000    0.9920        62

    accuracy                         0.8992       873
   macro avg     0.9212    0.9206    0.9177       873
weighted avg     0.9044    0.8992    0.8985       873



In [11]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Training Time: "+str(end-start))

Accuracy: 0.8991981672394044
Training Time: 3.5866281986236572
